In [784]:
import gensim
from collections import namedtuple
from collections import OrderedDict
import multiprocessing
import numpy as np
import pandas as pd
import json
import logging
import random
import pymongo
import nltk.data
from itertools import chain
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from scipy.spatial.distance import cdist
from nltk import pos_tag
from sklearn.metrics.pairwise import linear_kernel
from pymongo import MongoClient
from random import sample
import statsmodels.api as sm
from gensim.models import Doc2Vec
from gensim.models import Word2Vec
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.doc2vec import TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec

# for timing
from contextlib import contextmanager
from timeit import default_timer
import time

from random import shuffle
import datetime


In [785]:
client = MongoClient()
db = client['tweets_db']
cursor = db.tweets.find()
count = 200000
tweets = []
for tweet in cursor:
    count -= 1
    try:
        if 'text' in tweet:
            
            td = {}
#             td['id'] = tweet['id']
            td['text'] = tweet['text']
            td['hashtags'] = tweet['entities']['hashtags'][0]['text']
            td['timestamp'] = tweet['created_at']
            tweets.append(td)

    except:
        continue
    if count <= 0:
        break
        

In [786]:
# pd.DataFrame(tweets).ix[21,'hashtags'][0]['text']


In [787]:
df = pd.DataFrame(tweets)
df.head()


,hashtags,text,timestamp
0,TVFightsLive,Gotta go with @maudegarrett on this! We only h...,Tue Apr 19 23:57:59 +0000 2016
1,ALDUBLoveOnCue,RT @GasgasAbelgass: THIS GIRL IS FULL OF EXCIT...,Wed Apr 20 00:03:34 +0000 2016
2,RelationshipGoals,RT @Cheyennelt: @Tscarmardo1 when she just sup...,Wed Apr 20 00:03:34 +0000 2016
3,Opines,IMT and SITRI Partner to Accelerate Developmen...,Wed Apr 20 00:03:34 +0000 2016
4,Bernie2016,RT @howserob: CBS early exit poll: #Bernie2016...,Wed Apr 20 00:03:35 +0000 2016


In [788]:
# Create a set of documents.
tweet_list = []
for text in df['text'].values:
    text = re.sub(r"#|@|http\S+", "", text).lower()
    tweet_list.append(text)


In [789]:
docs = [word_tokenize(content) for content in tweet_list]
stop = set(stopwords.words('english'))
docs = [[word for word in words if word not in stop] for words in docs]
# snowball = SnowballStemmer('english')
# docs_snowball = [[snowball.stem(word) for word in words] for words in docs]
wordnet = WordNetLemmatizer()
docs_wordnet = [[wordnet.lemmatize(word) for word in words] for words in docs]
docs = docs_wordnet


In [790]:
# Part of speech tagging
# pos_tagged = [pos_tag(tokens) for tokens in docs]


In [794]:
docs_joined = [' '.join(x) for x in docs]
vectorizer = TfidfVectorizer(use_idf=False)
vectorized = vectorizer.fit_transform(docs_joined)
words = vectorizer.get_feature_names()
docs_vectorized = vectorizer.fit_transform(docs_joined).toarray()
cosine_similarities = linear_kernel(vectorized)


In [795]:
print np.argsort(cosine_similarities[0,:])[::-1]
print tweet_list[0]
print tweet_list[27247]
print tweet_list[5968]

[    0 27247  5968 ..., 24996 24995 18992]
gotta go with maudegarrett on this! we only have a few seasons left!i want a crazy epic character team up! 

tvfightslive screenjunkies
rt iheartradio: you don't gotta go to work, but you gotta put in work vaguejobdescriptions fifthharmony 
gorl...... they're both annoying, they gotta go bgc


In [796]:
print np.argsort(cdist(docs_vectorized[0, :][np.newaxis, :], docs_vectorized, 'cosine'))
print tweet_list[0]
print tweet_list[5968]
print tweet_list[11237]

[[    0  5968 11237 ...,  8630  8639 24983]]
gotta go with maudegarrett on this! we only have a few seasons left!i want a crazy epic character team up! 

tvfightslive screenjunkies
gorl...... they're both annoying, they gotta go bgc
rt fabulousfairy: it's a battle in my house, but i've gotta go with voicesaveowen


In [811]:
print np.argsort(cdist(docs_vectorized[0, :][np.newaxis, :], docs_vectorized, 'jaccard'))
print tweet_list[0]
print tweet_list[8510]
print tweet_list[10764]
print tweet_list[8114]
print tweet_list[9323]
print tweet_list[9838]

[[    0  8510 10764 ...,  8346  8369 24983]]
gotta go with maudegarrett on this! we only have a few seasons left!i want a crazy epic character team up! 

tvfightslive screenjunkies
rt brookecarbo: as much as love emily, and i do, gotta go voicesaveowen on this one... just adore that voice thevoice
rt brookecarbo: as much as love emily, and i do, gotta go voicesaveowen on this one... just adore that voice thevoice
rt brookecarbo: as much as love emily, and i do, gotta go voicesaveowen on this one... just adore that voice thevoice
rt eloramaxwell: gotta go with emily she takes me to another time and her voice is hauntingly beautiful!! ❤❤voicesaveemily
rt brookecarbo: as much as love emily, and i do, gotta go voicesaveowen on this one... just adore that voice thevoice


In [799]:
print np.argsort(cdist(docs_vectorized[0, :][np.newaxis, :], docs_vectorized, 'hamming'))
print tweet_list[0]
print tweet_list[23816]
print tweet_list[12694]

[[    0 23816 12694 ...,  1035  1037   295]]
gotta go with maudegarrett on this! we only have a few seasons left!i want a crazy epic character team up! 

tvfightslive screenjunkies
cuandolluevemedanganasde c o  m e r.
its on!!!! containment


In [801]:
print np.argsort(cdist(docs_vectorized[0, :][np.newaxis, :], docs_vectorized, 'euclidean'))
print tweet_list[0]
print tweet_list[5968]
print tweet_list[8246]

[[    0  5968  8246 ...,  4671 15366  6605]]
gotta go with maudegarrett on this! we only have a few seasons left!i want a crazy epic character team up! 

tvfightslive screenjunkies
gorl...... they're both annoying, they gotta go bgc
rt ktnieb: gotta go with owen because adamlevine asked me to voicesaveowen


In [804]:
print np.argsort(cdist(docs_vectorized[0, :][np.newaxis, :], docs_vectorized, 'seuclidean', V=None))
print tweet_list[0]
print tweet_list[7879]
print tweet_list[7881]

[[    0  7879  7881 ..., 20606   436 13843]]
gotta go with maudegarrett on this! we only have a few seasons left!i want a crazy epic character team up! 

tvfightslive screenjunkies
rt to voicesaveowen 
rt to voicesaveowen 


In [807]:
print np.argsort(cdist(docs_vectorized[0, :][np.newaxis, :], docs_vectorized, 'sqeuclidean'))
print tweet_list[0]
print tweet_list[5968]
print tweet_list[17081]

[[    0  5968 17081 ..., 20501 20556  9450]]
gotta go with maudegarrett on this! we only have a few seasons left!i want a crazy epic character team up! 

tvfightslive screenjunkies
gorl...... they're both annoying, they gotta go bgc
if you gotta girlfriend saythat


In [768]:
def most_similar(lst, tweets):
    '''
    IN: LIST, INTEGER, LIST
    OUT: LIST
    Return the tweets for indices with the highest n values.
    '''
    return [tweets[i] for i in np.argsort(lst)[-1:-n-1:-1]]
